# How to determine causality? A hotel booking demand study 

In [6]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns  
import dowhy
import os

### Data pre-processing

In [7]:
data = pd.read_csv('https://raw.githubusercontent.com/Sid-darthvader/DoWhy-The-Causal-Story-Behind-Hotel-Booking-Cancellations/master/hotel_bookings.csv')
data.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


### Data Dictionary

|variable                       |class     |description |
|:------------------------------|:---------|:-----------|
|hotel                          |string    | Resort Hotel or City Hotel |
|is_canceled                    |binary    | Indicating whether the booking was canceled, 1 - yes, 0 - no |
|lead_time                      |double    | Number of days between the booking and arrival dates |
|arrival_date_year              |double    | Year of arrival date|
|arrival_date_month             |string    | Month of arrival date|
|arrival_date_week_number       |double    | Week number of year for arrival date|
|arrival_date_day_of_month      |double    | Day of arrival date|
|stays_in_weekend_nights        |double    | Number of weekend nights booked |
|stays_in_week_nights           |double    | Number of week nights booked|
|adults                         |double    | Number of adults|
|children                       |double    | Number of children|
|babies                         |double    | Number of babies |
|meal                           |string    | Type of meal booked in meal packages: <br> Undefined/SC – no meal package;<br>BB – Bed & Breakfast; <br> HB – Half board (breakfast and one other meal – usually dinner); <br> FB – Full board |
|country                        |string    | Country of origin. Categories are represented |
|market_segment                 |string    | Market segment designation, “TA” - “Travel Agents”, “TO” - “Tour Operators” |
|distribution_channel           |string    | Booking distribution channel, “TA” - “Travel Agents”, “TO” - “Tour Operators” |
|is_repeated_guest              |binary    | Value indicating if the booking name was from a repeated guest 1 - yes, 0 - no |
|previous_cancellations         |double    | Number of previous bookings that were cancelled by the customer in the past |
|previous_bookings_not_canceled |double    | Number of previous bookings that were not cancelled by the customer in the past |
|reserved_room_type             |string    | Code of room type reserved |
|assigned_room_type             |string    | Code for the type of room assigned to the booking |
|booking_changes                |double    | Number of changes made to the booking from the moment the booking was entered into the system until the moment of check-in/cancellation |
|deposit_type                   |string    | Indication on if the customer made a deposit to guarantee the booking. <br>No Deposit – no deposit was made;<br>Non Refund – a deposit was made in the value of the total stay cost;<br>Refundable – a deposit was made with a value under the total cost of stay |
|agent                          |double    | ID of the travel agency that made the booking |
|company                        |double    | ID of the company that made the booking or responsible for paying the booking |
|days_in_waiting_list           |double    | Number of days the booking was in the waiting list before it was confirmed |
|customer_type                  |string    | Type of booking, <br>Contract - when the booking has an allotment or other type of contract associated to it;<br>Group – when the booking is associated to a group;<br>Transient – when the booking is not part of a group or contract, and is not associated to other transient booking;<br>Transient party – when the booking is transient, but is associated to at least other transient booking|
|adr                            |double    | Average Daily Rate as defined by dividing the sum of lodging transactions by the number of staying nights |
|required_car_parking_spaces    |double    | Number of car parking spaces required by the customer |
|total_of_special_requests      |double    | Number of special requests made by the customer (twin bed or high floor ext..)|
|reservation_status             |string    | Reservation last status<br>Canceled – booking was canceled by the customer;<br>Check-Out – customer has checked in but already departed;<br>No-Show – customer did not check-in and did inform the hotel of the reason why |
|reservation_status_date        |double    | Date at which the last status was set |

#### Clean null values

In [8]:
data.isnull().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

In [9]:
# Replacing children's null values by 0
data['children']= data['children'].fillna(0)

# Replacing missing countries with most freqently occuring countries
data['country']= data['country'].fillna('PRT')

# We will drop 'agent' and 'company' since we have no use in them
data = data.drop(['agent', 'company'], axis=1)

data.isnull().sum()

hotel                             0
is_canceled                       0
lead_time                         0
arrival_date_year                 0
arrival_date_month                0
arrival_date_week_number          0
arrival_date_day_of_month         0
stays_in_weekend_nights           0
stays_in_week_nights              0
adults                            0
children                          0
babies                            0
meal                              0
country                           0
market_segment                    0
distribution_channel              0
is_repeated_guest                 0
previous_cancellations            0
previous_bookings_not_canceled    0
reserved_room_type                0
assigned_room_type                0
booking_changes                   0
deposit_type                      0
days_in_waiting_list              0
customer_type                     0
adr                               0
required_car_parking_spaces       0
total_of_special_requests   

#### Merge and remove unnecessary columns

In [11]:
# total number of guests instead of the adults/children/babies separation
data['num_guests'] = (data['adults'] + data['children'] + data['babies'])
data = data.astype({'num_guests': int})

# total stay instead of the week night and weekend separation
data['total_stay'] = data['stays_in_week_nights'] + data['stays_in_weekend_nights']

# binary variable 'has_room_changed" instead of reserved room type and assigned room type
data['has_room_changed'] = 0
slice_indices = data['reserved_room_type'] != data['assigned_room_type']
data.loc[slice_indices,'has_room_changed'] = 1

# Deleting unnecessary attributes
data = data.drop(['adults', 'children', 'babies', 'stays_in_week_nights', 'stays_in_weekend_nights',
                  'reserved_room_type', 'assigned_room_type', 'reservation_status_date',
                  'market_segment', 'distribution_channel'], axis=1)
data.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,meal,country,is_repeated_guest,...,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,num_guests,total_stay,has_room_changed
0,Resort Hotel,0,342,2015,July,27,1,BB,PRT,0,...,No Deposit,0,Transient,0.0,0,0,Check-Out,2,0,0
1,Resort Hotel,0,737,2015,July,27,1,BB,PRT,0,...,No Deposit,0,Transient,0.0,0,0,Check-Out,2,0,0
2,Resort Hotel,0,7,2015,July,27,1,BB,GBR,0,...,No Deposit,0,Transient,75.0,0,0,Check-Out,1,1,1
3,Resort Hotel,0,13,2015,July,27,1,BB,GBR,0,...,No Deposit,0,Transient,75.0,0,0,Check-Out,1,1,0
4,Resort Hotel,0,14,2015,July,27,1,BB,GBR,0,...,No Deposit,0,Transient,98.0,0,1,Check-Out,2,2,0


### Exploratory data analysis